### File Discovery

* ftp://rockyftp.cr.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/USGS_LPC_LA_UpperDeltaPlain_2017_LAS_2018/

This notebook is for finding a list of relavent  elevation files for New Orleans, using the metatdata supplied

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import geoplot as gplt
import matplotlib.pyplot as plt
from os import listdir
from os.path import exists

import xml.etree.ElementTree as et

In [2]:
newOrleansBounds = {}
newOrleansBounds['north'] = 30.16
newOrleansBounds['south'] = 29.89
newOrleansBounds['east'] = -90.286
newOrleansBounds['west'] = -89.867

metatDataPath = './USGS_LPC_LA_UpperDeltaPlain_2017_LAS_2018/metadata/'
shapeFilePath = './USGS_LPC_LA_UpperDeltaPlain_2017_LAS_2018/las/tiled/'

In [3]:
def getBounds(name, elem):
    return float(elem.find(name).text)

def metaDataInBounds(xmlFileName, debug=False, bounds = newOrleansBounds, forceXmlBounds = False):
    tree = et.parse(xmlFileName)
    treeRoot = tree.getroot()
    idinfoElem = treeRoot.find('idinfo')
    spdomElem = idinfoElem.find('spdom')
    boundsElem = spdomElem.find('bounding')
    
    fileElem = treeRoot
    targetPath = ['distinfo','stdorder', 'digform', 'digtopt', 'onlinopt', 'computer','networka','networkr']
    for target in targetPath:
        if debug:
            print("targetPath: ", fileElem.tag)
            
        fileElem = fileElem.find(target)
        
    ftpFileName = fileElem.text
    
    
    if forceXmlBounds:
        (northbc,southbc,eastbc,westbc) = forceXmlBounds
    else:
        (northbc,southbc,eastbc,westbc) = (getBounds(name, boundsElem) for name in ['northbc','southbc','eastbc','westbc']  )
    
    overLap = True
    eastOverLap = True
    northOverLap = True
    #xml is east or west of bounds
    if eastbc < bounds['east'] or westbc > bounds['east']:
        overLap = False
        eastOverLap = False
        
    #xml is north or south  bounds
    if southbc > bounds['north'] or northbc < bounds['south']:
        overLap = False
        northOverLap = False
        
    if debug:
        print(northbc,southbc,eastbc,westbc)
        print("New Orleans bounds: ", bounds)
        print("OverLapped: ", overLap)
        print("FTP Filename: ", ftpFileName)
        
    return {
                'filename': xmlFileName.split('/')[-1],
                'overLap': overLap,
                'eastOverLap': eastOverLap,
                'northOverLap': northOverLap,
                'northbc': northbc,
                'southbc': southbc,
                'eastbc': eastbc,
                'westbc': westbc,
                'ftpFileName': ftpFileName
            }


In [4]:
testFileName = './USGS_LPC_LA_UpperDeltaPlain_2017_LAS_2018/metadata/USGS_LPC_LA_UpperDeltaPlain_2017_15rxp735285_LAS_2018_meta.xml'
metaDataInBounds(testFileName, True)

targetPath:  metadata
targetPath:  distinfo
targetPath:  stdorder
targetPath:  digform
targetPath:  digtopt
targetPath:  onlinopt
targetPath:  computer
targetPath:  networka
30.0888715084389 30.0751276331772 -91.1840808992984 -91.199884351646
New Orleans bounds:  {'north': 30.16, 'south': 29.89, 'east': -90.286, 'west': -89.867}
OverLapped:  False
FTP Filename:  ftp://rockyftp.cr.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/USGS_LPC_LA_UpperDeltaPlain_2017_LAS_2018/las/tiled/USGS_LPC_LA_UpperDeltaPlain_2017_15rxp735285_LAS_2018.zip


{'filename': 'USGS_LPC_LA_UpperDeltaPlain_2017_15rxp735285_LAS_2018_meta.xml',
 'overLap': False,
 'eastOverLap': False,
 'northOverLap': True,
 'northbc': 30.0888715084389,
 'southbc': 30.0751276331772,
 'eastbc': -91.1840808992984,
 'westbc': -91.199884351646,
 'ftpFileName': 'ftp://rockyftp.cr.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/USGS_LPC_LA_UpperDeltaPlain_2017_LAS_2018/las/tiled/USGS_LPC_LA_UpperDeltaPlain_2017_15rxp735285_LAS_2018.zip'}

In [5]:
xmlFiles = pd.DataFrame( [metatDataPath + name for name in listdir('./USGS_LPC_LA_UpperDeltaPlain_2017_LAS_2018/metadata/') if name[-4:] == '.xml'], columns=['filename'] )
xmlFiles = pd.DataFrame( xmlFiles['filename'].apply(metaDataInBounds).to_list() )

In [6]:
xmlFiles.head()

,filename,overLap,eastOverLap,northOverLap,northbc,southbc,eastbc,westbc,ftpFileName
0,USGS_LPC_LA_UpperDeltaPlain_2017_15ryp500150_L...,False,False,True,29.953899,29.940070,-90.394035,-90.409914,ftp://rockyftp.cr.usgs.gov/vdelivery/Datasets/...
1,USGS_LPC_LA_UpperDeltaPlain_2017_15ryp230195_L...,False,False,True,29.999679,29.985880,-90.672650,-90.688503,ftp://rockyftp.cr.usgs.gov/vdelivery/Datasets/...
2,USGS_LPC_LA_UpperDeltaPlain_2017_15ryp170585_L...,False,False,False,30.352434,30.338638,-90.726747,-90.742653,ftp://rockyftp.cr.usgs.gov/vdelivery/Datasets/...
3,USGS_LPC_LA_UpperDeltaPlain_2017_15ryp170555_L...,False,False,False,30.325382,30.311587,-90.727372,-90.743274,ftp://rockyftp.cr.usgs.gov/vdelivery/Datasets/...
4,USGS_LPC_LA_UpperDeltaPlain_2017_15ryp995270_L...,False,False,True,30.050959,30.037075,-89.878328,-89.894287,ftp://rockyftp.cr.usgs.gov/vdelivery/Datasets/...


In [7]:
np.count_nonzero(xmlFiles['northOverLap'])

1747

In [8]:
np.count_nonzero(xmlFiles['eastOverLap'])

46

In [9]:
np.count_nonzero(xmlFiles['overLap'])

14

In [10]:
for fileName in xmlFiles[xmlFiles['overLap'] == True]['ftpFileName']:
    localFile = fileName.split('/')[-1]
    if exists(shapeFilePath + localFile):
        print(localFile + " exists, skipping")
        
    else:
        print(localFile + " not found, downloading")
        !curl -O $fileName
        !mv $localFile $shapeFilePath

USGS_LPC_LA_UpperDeltaPlain_2017_15RYP605240_LAS_2018.zip exists, skipping
USGS_LPC_LA_UpperDeltaPlain_2017_15ryp620120_LAS_2018.zip exists, skipping
USGS_LPC_LA_UpperDeltaPlain_2017_15RYP605255_LAS_2018.zip exists, skipping
USGS_LPC_LA_UpperDeltaPlain_2017_15ryp605135_LAS_2018.zip exists, skipping
USGS_LPC_LA_UpperDeltaPlain_2017_15RYP605195_LAS_2018.zip exists, skipping
USGS_LPC_LA_UpperDeltaPlain_2017_15RYP605165_LAS_2018.zip exists, skipping
USGS_LPC_LA_UpperDeltaPlain_2017_15ryp605270_LAS_2018.zip exists, skipping
USGS_LPC_LA_UpperDeltaPlain_2017_15RYP605150_LAS_2018.zip exists, skipping
USGS_LPC_LA_UpperDeltaPlain_2017_15RYP605225_LAS_2018.zip exists, skipping
USGS_LPC_LA_UpperDeltaPlain_2017_15RYP605210_LAS_2018.zip exists, skipping
USGS_LPC_LA_UpperDeltaPlain_2017_15ryp620090_LAS_2018.zip exists, skipping
USGS_LPC_LA_UpperDeltaPlain_2017_15ryp620105_LAS_2018.zip exists, skipping
USGS_LPC_LA_UpperDeltaPlain_2017_15RYP605180_LAS_2018.zip exists, skipping
USGS_LPC_LA_UpperDeltaPla

In [12]:
xmlFiles.to_pickle('./xmlFiles.pickle')

In [14]:
!ls -lsah xmlFiles.pickle

1.4M -rw-r--r-- 1 g g 1.4M May  1 16:50 xmlFiles.pickle
